In [1]:
import json
import requests

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
from tqdm.notebook import tqdm

## Open State data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Poplation Data

In [19]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [20]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [21]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [22]:
states_with_population_df = states_with_population_df.dropna()

## Get Occupation Data

In [9]:
url = "https://data.bls.gov/OESServices/resultsgeoocc"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,ru;q=0.8,el;q=0.7",
    "cache-control": "no-cache",
    "content-type": "application/json;charset=UTF-8",
    "pragma": "no-cache",
    "sec-ch-ua": '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "Referer": "https://data.bls.gov/oes/",
}

In [10]:
body = {
    "areaTypeCode": "S",
    "areaCode": "0600000",
    "industryCode": "000000",
    "occupationCode": ["332011"],
    "datatype": ["xxxxxx"],
    "releaseDateCode": ["2024A01", "2024A01"],
    "outputType": "H",
}
r = requests.post(url, headers=headers, data=json.dumps(body))

In [11]:
r.json()

{'resultsGeoOccVO': [{'releaseDateCode': '2024A01',
   'releaseDate': 'May 2024',
   'footnotes': [{'footnote': '1',
     'text': 'Estimates for detailed occupations do not sum to the totals because the totals include occupations not shown separately. Estimates do not include self-employed workers.'},
    {'footnote': '2',
     'text': 'Annual wages have been calculated by multiplying the corresponding hourly wage by 2,080 hours.'},
    {'footnote': '3',
     'text': 'The relative standard error (RSE) is a measure of the reliability of a survey statistic. The smaller the relative standard error, the more precise the estimate.'}],
   'occupations': [{'occupationCode': '332011',
     'occupationName': 'Firefighters',
     'values': [{'dataTypeCode': '01', 'value': '       27580'},
      {'dataTypeCode': '02', 'value': '         1.9'},
      {'dataTypeCode': '03', 'value': '       47.21'},
      {'dataTypeCode': '04', 'value': '       98200'},
      {'dataTypeCode': '05', 'value': '      

In [12]:
data = []
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    body = {
        "areaTypeCode": "S",
        "areaCode": f"{states_df.iloc[i]['STATEFP']}00000",
        "industryCode": "000000",
        "occupationCode": ["332011"],
        "datatype": ["03", "08"],
        "releaseDateCode": ["2024A01", "2024A01"],
        "outputType": "H",
    }
    r = requests.post(url, headers=headers, data=json.dumps(body))
    json_resp = r.json()
    if json_resp["resultsGeoOccVO"][0]["occupations"]:
        hourly_wage = float(
            r.json()["resultsGeoOccVO"][0]["occupations"][0]["values"][0]["value"]
        )
        median_wage = float(
            r.json()["resultsGeoOccVO"][0]["occupations"][0]["values"][1]["value"]
        )
        data.append(
            {
                "STUSPS": states_df.iloc[i]["STUSPS"],
                "wage": hourly_wage,
                "median_wage": median_wage,
            }
        )
    else:
        print(states_df.iloc[i]["NAME"])

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

American Samoa
United States Virgin Islands
Hawaii
Guam
Commonwealth of the Northern Mariana Islands


In [13]:
occupation_df = pd.DataFrame(
    data,
)

## Merge Data

In [24]:
occupation_gdf = states_with_population_df.merge(occupation_df, on="STUSPS", how="left")

In [26]:
occupation_gdf = occupation_gdf.to_crs(9311)
occupation_gdf.to_file("data/Firefighters_Hourly_Wage_Per_State.gpkg")